In [1]:
%load_ext autoreload
%autoreload 2

import os
ROOT_DIR = os.getcwd()[:os.getcwd().rfind('NVcenter')]+ 'NVcenter'
os.chdir(ROOT_DIR)

In [2]:
from NVcenter import *
import numpy as np

In [3]:
# targets
H_gate = q.tensor( q.qeye(2), 1/np.sqrt(2) * (q.sigmax() + q.sigmaz()) )
CNOT_gate = q.tensor( q.fock_dm(2,0), q.qeye(2) ) + q.tensor( q.fock_dm(2,1), q.sigmax() )
init_state = q.tensor( q.fock_dm(2,0), q.fock_dm(2,0) )
H_state = H_gate * init_state * H_gate.dag()
CNOT_state = CNOT_gate * init_state * CNOT_gate.dag()

# C13 spins  with the correct factor 2pi
Suter_C13_1 = (4.722331100730915e-10, 0.0, 1.030637866442101e-10)
# Suter_C13_2 = (3.6747967115510627e-10, 0.0, 1.4217244863175118e-10)
# Suter_C13_3 = (4.1283485801972827e-10, 0.0, 8.964851198130749e-11)
# Suter_C13_4 = (3.6919326714940366e-10, 0.0, 8.849893347694991e-11)

# pulse sequences
# pulse_seq = [0e-6, 4.06e-6, 1.57e-6, 1.51e-6, 1.00, 3.58, 1.68, 0.69, 1.97, 0.50] # Dominik Hadamard
# pulse_seq = [0.74e-6, 0.22e-6, 0.43e-6, 0.89e-6, 0.23e-6, 1.26e-6, 1.50e-6, 3*np.pi/2, 3*np.pi/2, np.pi/2] # Suter Hadamard 
# pulse_seq = [3.78e-6, 2.11e-6, 2.15e-6, 0.63e-6, 1.88e-6, 3.96e-6, 1.9e-6, 0, np.pi/5, np.pi/2] # Suter CNOT

In [4]:
register_config = [('NV', (0, 0, 0), 0, {}), ('C13', Suter_C13_1, 0, {})]
bath_config = SpinBath('C13', 0.02e-2, 2e-9, 4.2e-9).config
approx_level = 'no_bath'
target = H_gate

pulse_seq = [0.74e-6, 0.22e-6, 0.43e-6, 0.89e-6, 0.23e-6, 1.26e-6, 1.50e-6, 3*np.pi/2, 3*np.pi/2, np.pi/2] # Suter Hadamard 
pulse = Pulse(pulse_seq, register_config, bath_config, approx_level, target, old_state=None, mode='unitary_gate' )

t_list = np.linspace(0, pulse.total_time, 20)
gate_list = [pulse.calc_new_states(t) for t in t_list]

fig, ax = plt.subplots()
ax.plot(t_list, [gate[0,0] for gate in gate_list])
ax.plot(t_list, [gate[0,1] for gate in gate_list])
ax.plot(t_list, [gate[0,2] for gate in gate_list])
ax.plot(t_list, [gate[0,3] for gate in gate_list])

IndexError: list index out of range

## Reinforcement Learning

In [113]:
class Environment:
    def __init__(self, register_config, bath_config, approx, target, dynamical_decoupling=False):

        self.register_config = register_config
        self.bath_config = bath_config
        self.approx = approx
        self.target = target
        self.dynamical_decoupling = dynamical_decoupling

        self.hamiltonian = Hamiltonian(self.register_config, self.bath_config, 'no_bath')
        self.register_init_state = self.hamiltonian.register_init_state
        
        # reset emvironment
        self.state = self.register_init_state
        self.fidelity = calc_fidelity(self.state, self.target)

    def reset(self):
        self.state = self.init_state
        self.fidelity = calc_fidelity(self.state, self.target)

    def step(self, pulse_seq):
        if self.approx == 'gCCE':
            self.state = self.get_new_states_gCCE(pulse_seq, self.state, 2)
        else: 
            self.state = self.get_new_states(pulse_seq, self.state, self.approx)[0]
            
        self.fidelity = calc_fidelity(self.state, self.target)
        return self.state, self.fidelity
        

    def get_new_states(self, pulse_seq, old_state, approx_level):
        pulse = Pulse(pulse_seq, 
                      self.register_config, 
                      self.bath_config, 
                      approx_level, 
                      self.target, 
                      old_state=old_state, 
                      dynamical_decoupling=self.dynamical_decoupling)

        if approx_level == 'gCCE2':
            return pulse.new_states, pulse.idx_gCCE2
        return pulse.new_states
    
    def get_new_states_gCCE(self, pulse_seq, old_states, gCCE_order):
        gCCE0_dm = self.get_new_states(pulse_seq, old_states, 'gCCE0')[0]
        gCCE_dm = gCCE0_dm
        
        if gCCE_order == 0:
            return gCCE_dm
            
        gCCE1_dms = self.get_new_states(pulse_seq, old_states, 'gCCE1')
        gCCE_dm_correction = np.prod( [gCCE1_dm.full() / gCCE0_dm.full() for gCCE1_dm in gCCE1_dms], axis=0 )
        gCCE_dm_correction = q.Qobj(gCCE_dm, dims=[[2, 2], [2, 2]])
        gCCE_dm *= gCCE_dm_correction
        
        if gCCE_order == 1:
            return gCCE_dm
            
        gCCE2_dms, idx_gCCE2 = self.get_new_states(pulse_seq, old_states, 'gCCE2') 
        gCCE_dm_correction = np.prod( [(gCCE2_dm.full() * gCCE0_dm.full()) / (gCCE1_dms[i].full() * gCCE1_dms[j].full()) for gCCE2_dm, (i,j) in zip(gCCE2_dms, idx_gCCE2)], axis=0 )
        gCCE_dm_correction = q.Qobj(gCCE_dm, dims=[[2, 2], [2, 2]])
        gCCE_dm *= gCCE_dm_correction
        
        if gCCE_order == 2:
            return gCCE_dm
            
        else:
            return NotImplementedError

In [115]:
register_config = [('NV', (0, 0, 0), 0, {}), ('C13', (0.87e-9, 0, 0.19e-9), 0, {})]
bath_config = SpinBath('C13', 0.02e-2, 2e-9, 2.5e-9).config
approx = 'full_bath'

pulse_seq = [0e-6, 4.06e-6, 1.57e-6, 1.51e-6, 1.00, 3.58, 1.68, 0.69, 1.97, 0.50]
target = 1/np.sqrt(2) * (q.tensor(q.fock_dm(2,0), q.fock_dm(2,0)) + q.tensor(q.fock_dm(2,1), q.fock_dm(2,1)) )

env = Environment(register_config, bath_config, approx, target, dynamical_decoupling=False)
env.step(pulse_seq)[1]

0.4355367060849788